# Lab 5: Bayesian Classification
### COSC 426: Fall 2025, Colgate University

## Part 1: Build a unigram model

#### Part 1.1

Start by understanding what is happening when you initilize a UnigramModel object

In [ ]:
from UnigramModel import UnigramModel
import util
import math
import pandas as pd
import numpy as np

sample_model = UnigramModel(tokenize = util.nltk_tokenize,
                            tokenizer_kwargs = {},
                            vocab = util.get_vocab('data/glove_vocab.txt'),
                            unk_token = '[UNK]',
                            train_paths = ['data/sample-alice.txt'],
                            smooth = 'add-0.1',
                            lower = True
                           )                    

/Users/jeong/Projects/cosc426-projects/cosc426/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**What are the parameters required to initialize a `UnigramModel` object and how are these parameters used in the `UnigramModel` class?**

#### Part 1.2

Verify your implementation of `get_prob` and `evaluate` with the code below. 

*Hint: Running into errors "Rabbit" in `get_probs`? Think carefully about when are where preprocessing is being applied in the pipeline, and what the expected input for `get_probs` is*

We need tokenizer, vocab(with unknown token), train data, smoothing stretegy.
They all have default values. Tokeniger is used to split text in to tokens. Vocab is used to mark whether a work is known or not. Train data is the text we use to find n-gram frequencies. `smooth` is used to indiciate the kind of smoothing strategy we use get the probability.

In [2]:
## print prob of words
expected_probs = {'rabbit': 0.00010176146615934851,
                  'it': 0.0002755005547240899,
                  '[UNK]': 0.00017622107554423767,
                  'Alice': 0.00017622107554423767,
                  'Rabbit': 0.00017622107554423767,
                  'maze': 2.4819869794963054e-06
                 }

for word in expected_probs:
    if sample_model.get_prob(word) != expected_probs[word]:
        print(word, '\t incorrect')
        print('expected', expected_probs[word])
        print('got', sample_model.get_prob(word))
        print()
        

## Create paths and then load it
sample_model.evaluate(datafpath= 'data/sample-alice.txt',
                      predfpath = 'predictions/my_sample_preds_alice.tsv')



correct_df = pd.read_csv('predictions/my_sample_preds_alice.tsv', sep='\t')
my_df = pd.read_csv('predictions/sample_preds_alice.tsv', sep='\t')


# Does element wise comparison
print('Are dfs same?', correct_df.equals(my_df))

Are dfs same? True


## Part 2: Implement building blocks of a Naive Bayesian Classifier

Implement the building blocks for a Bayesian classifier. Here is a function that might be useful. 

### Part 2.1: Describe your approach

p(class | text) ~ p(text | class) * p(class)

How is unigram model related to likelihood?  
p(text | class) is probability of the word from the unigram model trained on the class text.

We can get p(text|class) by useing the unigram model trained on the class text, then we calculate p(class) using the 1/number of class. When we multiply the `likelihood` and the `prior`, we get the probability of the class given text.

### Part 2.2 Implement functions

In [3]:
import pandas as pd
def summarize(fname:str, aggregrate_type:str, aggregrate_col:str, groupby_cols:list, delimiter='\t'):
    """
    Args:
        fname: fpath to tsv/ csv file
        aggregate_type: mean or sum

        aggrefate_col: the column with values you want to aggregate over

        groupby_cols: the columns with the groups. 

    Returns:
        Pandas Dataframe with as many rows as unique group combinations. The values of rows in each group is either summed together or averaged depending on the aggregate_type. 

    """
    dat = pd.read_csv(fname, sep=delimiter)

    summ = dat.groupby(groupby_cols).agg({aggregrate_col: aggregrate_type}).reset_index()

    return summ

#### **Calculating the likelihood**

Start by calculating the likelihood of some text given models trained on text from different classes --- i.e., $P(text \mid model=class1)$, $P(text \mid model=class2)$, etc

*Hint: Think about why `summarize` function provided is useful* 

In [4]:
def get_likelihood(models_dict:dict, eval_fpath, class_label):
    """
    Params:
        models_dict: keys are classes and values are the models trained on the classes
        eval_fpath: the file models should be evaluated on
        class_label: the correct class label for sequences in the file 

    Returns:
        A Dataframe with the following columns: 
            sentid: id of the sentence
            model: the model being used to generate the likelihood
            likelihood: the sum of log probability across all the words in the sequence
            target_class: same as class_label
        
    """

    text = models_dict[class_label].preprocess([eval_fpath])
    rows = []
    for model, um in models_dict.items():
        for sentid in range(len(text)):
            sent = text[sentid]
            if sent:
                    likelihood = 0
                    for wordpos in range(len(sent)):
                        word = sent[wordpos]
                        prob = um.get_prob(word)
                        surp = math.log2(prob)
                        likelihood += surp
                    rows.append([sentid, model, likelihood, class_label])

    # df = pd.DataFrame(rows, columns=["sentid", "model", "likelihood", "class_label"])
    df = pd.DataFrame(rows, columns=["sentid", "model", "likelihood", "target_class"])
    return df

Once you've implemented this function, verify that your output matches the expected output below. 

In [5]:
import numpy as np
sample_models = {
    'alice': UnigramModel(tokenize = util.nltk_tokenize,
                            tokenizer_kwargs = {},
                            vocab = util.get_vocab('data/glove_vocab.txt'),
                            unk_token = '[UNK]',
                            train_paths = ['data/sample-alice.txt'],
                            smooth = 'add-0.1',
                            lower = True
                           ),
    'sherlock': UnigramModel(tokenize = util.nltk_tokenize,
                            tokenizer_kwargs = {},
                            vocab = util.get_vocab('data/glove_vocab.txt'),
                            unk_token = '[UNK]',
                            train_paths = ['data/sample-sherlock.txt'],
                            smooth = 'add-0.1',
                            lower = True
                           )
}

my_df = get_likelihood(sample_models, 'data/sample-lookingglass.txt', 'alice').reset_index(drop=True)
correct_df = pd.read_csv('predictions/sample-likelihood.tsv', sep='\t').reset_index(drop=True)

# using this instead of equal because of floating point imprecision
print('Printing proportion of matched values across correct_df and my_df\n')
print('likelihood', np.isclose(my_df['likelihood'], correct_df['likelihood']).sum()/len(my_df)) 
for col in ['sentid', 'model', 'target_class']:
    print(col, (my_df[col] == correct_df[col]).sum()/len(my_df))

Printing proportion of matched values across correct_df and my_df

likelihood 1.0
sentid 1.0
model 1.0
target_class 1.0


#### Calculating the prior

In [6]:
def get_prior(data: dict, tokenizer) -> dict:
    """
    Args:
        data: dictionary where keys are the classes, and values are filepaths to the class specific data

    Returns:
        Dictionary with prior probability for each class, which is the number of words in the class divided by the total number of words across all classes. 
        What counts as a word is determined by the tokenizer

    """
    n = 0 # total word count
    cwc = {} # calss word count: [class]: num
    for cls, files in data.items():
        count = 0
        cwc[cls] = 0

        for f in files:
            with open(f, "r") as file:
                text = file.readlines()

        for sent in text:
            curr_dat = tokenizer(sent, {})
            count += len(curr_dat)
            cwc[cls] += len(curr_dat)

        n += count

    # compute prior probabilities
    prior = {key: val / n for key, val in cwc.items()}
    return prior


In [7]:
dat_dict = {'sherlock': ['data/sample-sherlock.txt'],
            'alice': ['data/sample-alice.txt']}

correct_prior = {'sherlock': 0.4423076923076923, 'alice': 0.5576923076923077}
get_prior(dat_dict, util.nltk_tokenize)  # should match correct_prior

{'sherlock': 0.4423076923076923, 'alice': 0.5576923076923077}

#### Compute posterior

*Hint: Think about why `summarize` function provided is useful*

In [8]:
def get_posterior(models_dict, eval_fpath, class_label, prior_dict):
    """
    Args:
        Dictionary where keys are classes and values are the models trained on the classes

        eval_fpath: the file models should be evaluated on. 

        class_label: the label of the file that models are evaluated on

        prior_dict: prior probabilities of classes

    Returns:
        A Dataframe with the following columns: sentid, model, likelihood, class. 

    If you set eval_fpath to sample_reviews_test_positive.txt, you should get a dataframe that looks like this. (Its ok if you end up having additional columns)

   sentid model     likelihood   class        prior      posterior
       0  positive  -100.975898  positive    -0.736966   -101.712864
       1  positive  -100.941133  positive    -0.736966   -101.678099
       0  negative  -101.938780  positive    -1.321928   -103.260708
       1  negative  -101.938780  positive    -1.321928   -103.260708


    """
    text = models_dict[class_label].preprocess(eval_fpath)
    rows = []
    for model, um in models_dict.items():
        for sentid in range(len(text)):
            sent = text[sentid]
            if sent:
                    likelihood = 0
                    for wordpos in range(len(sent)):
                        word = sent[wordpos]
                        prob = um.get_prob(word)
                        surp = math.log2(prob)
                        likelihood += surp
                    prior = math.log2(prior_dict[model])
                    posterior = likelihood + prior
                    rows.append([sentid, model, likelihood, class_label, prior, posterior])

    df = pd.DataFrame(rows, columns=["sentid", "model", "likelihood", "class", "prior", "posterior"])
    return df

In [9]:
my_df = get_posterior(
    sample_models,
    ["data/sample-lookingglass.txt"],
    "alice",
    get_prior(dat_dict, util.nltk_tokenize),
).reset_index(drop=True)

correct_df = pd.read_csv('predictions/sample-posterior.tsv', sep='\t').reset_index(drop=True)
print('posterior', np.isclose(my_df['posterior'], correct_df['posterior']).sum()/len(my_df)) 

posterior 1.0


#### Implement classify

In [10]:
def classify(posterior):
    """
    Args:
        Dataframe with posterior probabilities

    Returns: 
        Dataframe where each sentence id is associated with a prediction. 
    """

    classes = posterior['model'].unique()
    wide_df = posterior.pivot(index=['sentid', 'class'],
                              columns=['model'],
                              values='posterior').reset_index()    
    wide_df['pred'] = wide_df[classes].idxmax(axis=1) # finding the highest class.
    return wide_df


In [11]:
posterior = get_posterior(sample_models, 
            ['data/sample-lookingglass.txt'],
            'alice',
             get_prior(dat_dict, util.nltk_tokenize)).reset_index(drop=True)
my_df = classify(posterior).reset_index(drop=True)

correct_df = pd.read_csv('predictions/sample-classify.tsv', sep='\t').reset_index(drop=True)
print('pred', (my_df['pred']==correct_df['pred']).sum()/len(my_df)) 



pred 1.0


#### Compute accuracy

In [12]:
def analyze(models_dict, eval_dict, prior_dict):
    """
    Args:
        models_dict: keys are classes, values are models trained on data from the class. 

        eval_dict: keys are classes, values are fpaths to evaluation data where the correct label is the class associated with the key

        prior_dict: keys are classes, values are prior probabilties of the classes. 

    Returns:
        Float which is the accuracy of the predictions across all classes

    """

    all_posteriors = []
    for cls, fpath in eval_dict.items():
        posterior = get_posterior(models_dict, fpath, cls, prior_dict)
        all_posteriors.append(posterior)

    all_posteriors_df = pd.concat(all_posteriors).reset_index(drop=True)
    classified_df = classify(all_posteriors_df).reset_index(drop=True)

    accuracy = (classified_df['pred'] == classified_df['class']).sum() / len(classified_df)

    return accuracy

In [13]:
sample_models = {
    'alice': UnigramModel(tokenize = util.nltk_tokenize,
                            tokenizer_kwargs = {},
                            vocab = util.get_vocab('data/glove_vocab.txt'),
                            unk_token = '[UNK]',
                            train_paths = ['data/sample-alice.txt'],
                            smooth = 'add-0.1',
                            lower = True
                           ),
    'sherlock': UnigramModel(tokenize = util.nltk_tokenize,
                            tokenizer_kwargs = {},
                            vocab = util.get_vocab('data/glove_vocab.txt'),
                            unk_token = '[UNK]',
                            train_paths = ['data/sample-sherlock.txt'],
                            smooth = 'add-0.1',
                            lower = True
                           )
}

## for simplicity making eval the same as train
sample_eval = {
    'alice': ['data/sample-lookingglass.txt'],
    'sherlock': ['data/sample-sherlock.txt']
}

sample_prior = get_prior({'sherlock': ['data/sample-sherlock.txt'],
                          'alice': ['data/sample-alice.txt']}, util.nltk_tokenize)


target_acc = 92.857

analyze(sample_models, sample_eval, sample_prior)


np.float64(0.9285714285714286)

## Part 3: Build Naive Bayesian Sentiment Classifier
Add as many code and markdown chunks as is helpful

In [14]:
import os

pos_dpath = 'data/aclImdb/train/pos'
neg_dpath = 'data/aclImdb/train/neg'

post_fpaths = [os.path.join(pos_dpath, fname) for fname in os.listdir(pos_dpath)]
neg_fpaths = [os.path.join(neg_dpath, fname) for fname in os.listdir(neg_dpath)]

imdb_models = {
    "pos": UnigramModel(
        tokenize=util.nltk_tokenize,
        tokenizer_kwargs={},
        vocab=util.get_vocab("data/aclImdb/imdb.vocab"),
        unk_token="[UNK]",
        train_paths=post_fpaths,
        smooth="add-0.1",
        lower=True,
    ),
    "neg": UnigramModel(
        tokenize=util.nltk_tokenize,
        tokenizer_kwargs={},
        vocab=util.get_vocab("data/aclImdb/imdb.vocab"),
        unk_token="[UNK]",
        train_paths=neg_fpaths,
        smooth="add-0.1",
        lower=True,
    ),
}

In [15]:
# Generate predictions from the model on the test set
pos_test_dpaths = "data/aclImdb/test/pos"
pos_test_fpaths = [
    os.path.join(pos_test_dpaths, fname)
    for fname in os.listdir(pos_test_dpaths)
]

neg_test_dpaths = "data/aclImdb/test/neg"
neg_test_fpaths = [
    os.path.join(neg_test_dpaths, fname)
    for fname in os.listdir(neg_test_dpaths)
]

imdb_eval = {
    "pos": pos_test_fpaths,
    "neg": neg_test_fpaths,
}

imdb_prior = get_prior(
    {"pos": pos_test_fpaths, "neg": neg_test_fpaths},
    util.nltk_tokenize,
)
print(imdb_prior)
analyze(imdb_models, imdb_eval, imdb_prior)

{'pos': 0.7658227848101266, 'neg': 0.23417721518987342}


np.float64(0.80708)

An 80.7% accuracy is significantly higher than that of a random guess (50%), so the model has successfully learned the positive and negative sentiments. This is quite surprising to me since we are using a simple unigram model that only relies on single words. My assumption as to what contributed to this high accuracy is that I used "add-0.1" as the smoothing method instead of a higher k-value. I did so because the evaluation dataset was from the same repository of text (movie critiques), so I was confident that the training and evaluation sets would be very similar.

However, there is a clear limit to the model, since it cannot account for a negated expression such as, "This movie was not a great, wonderful, brilliant film." Since there are more positive words than negative words, it would certainly evaluate the sentence as positive.

## Part 4 (optional): Build Bigram Bayesian Sentiment Classifier
Add as many code and markdown chunks as is helpful